In [1]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [2]:
filename = "../data/05/ABox.ttl"

In [3]:
text1 = '''CQ_2.1
What are the identifiers of the printed volumes? What are their types?
'''

query1 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?resource ?identifier_text ?type
WHERE {
    ?resource a frbroo:F4_Manifestation_Singleton ;
                crm:P2_has_type aat:300265632 ;
                crm:P48_has_preferred_identifier ?identifier .
    ?identifier crm:P2_has_type ?type ;
                crm:P190_has_symbolic_content ?identifier_text.
}
'''

In [4]:
text2 = '''CQ_2.2
What is the shelf mark of the cultural resource with the project ID "45"?
'''

query2 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?identifier_text
WHERE {
    ?resource a frbroo:F4_Manifestation_Singleton ;
    crm:P48_has_preferred_identifier ?identifier1 ,
                      ?identifier2 .
    ?identifier1 crm:P2_has_type aat:300312355 ;
                 crm:P190_has_symbolic_content "45" .
    ?identifier2 crm:P2_has_type aat:300404704 ;
                 crm:P190_has_symbolic_content ?identifier_text.
}
'''

In [5]:
text3 = '''CQ_2.3
What are the descriptive labels of the cultural resources that either have shelf marks or are prints?
'''

query3 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?resource ?label
WHERE {
    ?resource a frbroo:F4_Manifestation_Singleton ;
                crm:P48_has_preferred_identifier ?identifier ;
                crm:P2_has_type ?type ;
                crm:P3_has_note ?label .
    ?identifier crm:P2_has_type ?id_type .
    FILTER(?type = aat:300041273 || ?id_type = aat:300404704)
}
'''

In [6]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3)]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_2.1
What are the identifiers of the printed volumes? What are their types?

+--------------------------------------------------+-------------------+-------------------------------------------+
| resource                                         | identifier_text   | type                                      |
|--------------------------------------------------+-------------------+-------------------------------------------|
| https://w3id.org/dharc/ontology/chad-ap/object/data/0205-man | 05            | http://vocab.getty.edu/page/aat/300312355 |
| https://w3id.org/dharc/ontology/chad-ap/object/data/0205-man | 9                 | http://vocab.getty.edu/page/aat/300445021 |
| https://w3id.org/dharc/ontology/chad-ap/object/data/0205-man | A.V.GG.VII.28     | http://vocab.getty.edu/page/aat/300404704 |
+--------------------------------------------------+-------------------+-------------------------------------------+
CQ_2.2
Return the `identifier` with `type` `shelf mark` of the `cultur